<a href="https://colab.research.google.com/github/Amelia-ZJ/DSC530/blob/main/DSC550_Exercise_9_2_AFarrell_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 9 Exercise 9.2

## Best Model Selection and Hyperparameter Tuning

Amelia Farrell

DSC550

2/3/25


In [ ]:

# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the google drive directory
%cd /content/drive/MyDrive/Colab\ Notebooks/DSC550

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/DSC550


1. Import the dataset and ensure that it loaded properly.

In [ ]:
# importing first libraries needed
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
# loading dataset as a data frame
df = pd.read_csv('Loan_Train.csv')
df.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


2. Prepare the data for modeling by performing the following steps:

*    Drop the column “Load_ID.”
*    Drop any rows with missing data.
*    Convert the categorical features into dummy variables.

In [ ]:
# Drop the "Loan_ID" column
df.drop(columns=['Loan_ID'], inplace=True)

# Drop any rows with missing data
df.dropna(inplace=True)

# Display the first few rows
df.head(5)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [ ]:
# Identify categorical features
categorical_cols = df.select_dtypes(include=['object']).columns

# Convert categorical features into dummy variables
df2 = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Display the first few rows of the transformed dataset
df2.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,True,True,True,False,False,False,False,False,False,False
2,3000,0.0,66.0,360.0,1.0,True,True,False,False,False,False,True,False,True,True
3,2583,2358.0,120.0,360.0,1.0,True,True,False,False,False,True,False,False,True,True
4,6000,0.0,141.0,360.0,1.0,True,False,False,False,False,False,False,False,True,True
5,5417,4196.0,267.0,360.0,1.0,True,True,False,True,False,False,True,False,True,True


3. Split the data into a training and test set, where the “Loan_Status” column is the target.

In [ ]:
# Convert all boolean columns to 0/1 for model prep
df3 = df2.astype(int)
df3.head(5)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
1,4583,1508,128,360,1,1,1,1,0,0,0,0,0,0,0
2,3000,0,66,360,1,1,1,0,0,0,0,1,0,1,1
3,2583,2358,120,360,1,1,1,0,0,0,1,0,0,1,1
4,6000,0,141,360,1,1,0,0,0,0,0,0,0,1,1
5,5417,4196,267,360,1,1,1,0,1,0,0,1,0,1,1


In [ ]:
# Define features (X) and target (y)
X = df3.drop(columns=['Loan_Status_Y'])  # Features
y = df3['Loan_Status_Y']  # Target variable

# Split the dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display the shapes of the splits
print(f"Training set: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Test set: X_test: {X_test.shape}, y_test: {y_test.shape}")

Training set: X_train: (384, 14), y_train: (384,)
Test set: X_test: (96, 14), y_test: (96,)


4. Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

In [ ]:
# Create a pipeline with MinMaxScaler and KNN classifier
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Normalize features
    ('classifier', KNeighborsClassifier(n_neighbors=5))  # KNN classifier with k=5 & Placeholder classifier
])

5. Fit a default KNN classifier to the data with this pipeline. Report the model accuracy on the test set. Note: Fitting a pipeline model works just like fitting a regular model.

In [ ]:
# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.7083


6. Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. (see section 15.3 in the Machine Learning with Python Cookbook).


In [ ]:
# Define search space for n_neighbors (1 to 10)
param_grid = {'classifier__n_neighbors': list(range(1, 11))}

7. Fit a grid search with your pipeline, search space, and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.


In [ ]:
# Set up GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1) # CV = 5: five fold cross validation

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                     10]},
             scoring='accuracy')

8. Find the accuracy of the grid search best model on the test set. Note: It is possible that this will not be an improvement over the default model, but likely it will be.

In [ ]:
# Print best parameters and best score
print(f"Best n_neighbors: {grid_search.best_params_['classifier__n_neighbors']}")
print(f"Best Accuracy: {grid_search.best_score_:.4f}")

# Evaluate on the test set
test_accuracy = grid_search.best_estimator_.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


Best n_neighbors: 4
Best Accuracy: 0.7498
Test Accuracy: 0.6667


9. Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values in section 12.3 of the Machine Learning with Python Cookbook.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create dictionary with candidate learning algorithms and their hyperparameters
search_space = [
    {"classifier": [KNeighborsClassifier()],
     "classifier__n_neighbors": list(range(1, 11))},

    {"classifier": [LogisticRegression(max_iter=500, solver='liblinear')],
     "classifier__penalty": ['l1', 'l2'],
     "classifier__C": np.logspace(0, 4, 10)},

    {"classifier": [RandomForestClassifier()],
     "classifier__n_estimators": [10, 100, 1000],
     "classifier__max_features": [1, 2, 3]}
]

# Create grid search with 5-fold cross-validation
gridsearch = GridSearchCV(pipeline, search_space, cv=5, verbose=0)

# Fit grid search
best_model = gridsearch.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifier',
                 LogisticRegression(max_iter=500, penalty='l1',
                                    solver='liblinear'))])


10. What are the best model and hyperparameters found in the grid search? Find the accuracy of this model on the test set.


In [ ]:
# Print best model and hyperparameters
print(best_model.best_estimator_)

# Evaluate on the test set
test_accuracy = best_model.best_estimator_.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifier',
                 LogisticRegression(max_iter=500, penalty='l1',
                                    solver='liblinear'))])
Test Accuracy: 0.8333


11. Summarize your results.

This analysis was done for data on Loan applications and their outcomes (approved/denied (Y/N). To start, we loaded the data and prepped it for modeling by dropping 'Loan_ID', replacing/removing missing values, and converting any categorical columns to dummy variables. We were then ready to split our data into training (80%) and test (20%) sets (Training set: 384, Test set: 96).

For modeling, first ran a KNN classifier (K=5) that gave us 70% accuracy on the training set. We then ran a grid search to find us the BEST n_neighbors value for a KNN classifier along with 5-fold cross-validation. K=4 and 5-fold cross-validation created a better performing model (75% accurate vs 70%). 5-fold cross-validation helped improve it's performance by splitting the data into 5 different training and validation sets, ensuring the model learns patterns that generalizes well to new data. The best n_neighbors found was 4, with the best cross-validation accuracy of 75% but only %67 accuracy on the test set.

While 67% accuracy is not terrible. Would another model preform better? To find out, we expanded our hyperparameter search to include two other classifiers, Logistic Regression and Random Forest. We did this by using GridSearchCV (5-fold cross-validation), which can systematically test our chosen classifiers along with their respective hyperparameter values. In order to do so, we created a dictionary with all three models and their hyperparameters and ran it through our pipeline. Through this test, we found that the best model was actually Logistic Regression model (83% accuracy) with a penalty of L1, liblinear Solver, and regularization parameter C.

But how/why was Logistic Regression chosen as the best? The GridSearchCV function was able to perform a 5-fold cross-validation on each combination of model and hyperparameters. It then evaluated each model based on accuracy, which was then used as the scoring metric. The model that achieved the highest cross-validation accuracy was then selected as the best model.
GridSearchCV was also able to test the selected model on the test set to ensure that it generalized well.

Logistic Regression likely outperformed the other two models because our data set is well structured and most of our features are numeric and binary. Logistic Regression works well with structured, linearly separable binary classification problems.

KNN did not do as well because of our binary features. Because KNN relies on distance metrics, it can struggle to find meaningful nearest neighbors between just two values.

Random Forest may have performed worse because it was overfitting due to the small dataset size. Random Forest needs sufficient data to generalize well.
